In [9]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
import dataloader as dl

In [10]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10
USE_CUDA = torch.cuda.is_available()
#DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", DEVICE)

다음 기기로 학습합니다: cuda


In [11]:
train_loader=dl.load_trainset("/home/dhk1349/Desktop/Capstone Design2/ntu/xsub/")
test_loader=dl.load_testset("/home/dhk1349/Desktop/Capstone Design2/ntu/xsub/")

In [ ]:
class LSTM(nn.Module):
    #def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
    def __init__(self, n_layers, hidden_dim, input_shape, n_classes, dropout_p=0.2):
        super(LSTM, self).__init__()
        print("Building Basic LSTM model...")
        self.n_layers = n_layers 
        #self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(input_shape, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        #x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        c_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.lstm(x, (h_0, c_0))  # [i, b, h]
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.out(h_t)  # [b, h] -> [b, o]
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [ ]:
def train(model, optimizer, train_iter):
    model.train()
    for batch in train_iter:
        x, y = np.array(batch[0][0]), batch[1]
        
        X=[]
        for i in range(10):
            X.append(x[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        
        X=torch.tensor(np.array(X)).unsqueeze(0).to(DEVICE)
        y=y.to(DEVICE)
        
        optimizer.zero_grad()

        logit = model(X)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

In [ ]:
    """
def train(model, optimizer, data_idx):
    model.train()
    for idx, i in enumerate(data_idx): #suppose no batch
        sample=data[i][...,0].transpose(1,0,2) #shape 300,3,25
        #print(sample.shape)
        
        x=[] #Devide 300frame into 10 sectors and choose 1 frame from each sector
        
        for i in range(10):
            x.append(sample[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        
        X=torch.tensor(np.array(x)).unsqueeze(0).to(DEVICE)
        y=torch.tensor(y_label[idx]).to(DEVICE)
        optimizer.zero_grad()
        logit = model(X)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()
    """ 
    

In [ ]:
def evaluate(model, val_iter):
    #evaluate model
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = np.array(batch[0][0]), batch[1]
        
        X=[]
        for i in range(10):
            X.append(x[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        
        X=torch.tensor(np.array(X)).unsqueeze(0).to(DEVICE)
        y=y.to(DEVICE)
        
        logit = model(X)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [ ]:
    """
def evaluate(model, val_iter):
    #evaluate model
    model.eval()
    corrects, total_loss = 0, 0
    for idx, i in enumerate(test_data_idx): #suppose no batch
        sample=test_data[i][...,0].transpose(1,0,2) #shape 300,3,25
        #print(sample.shape)
        
        x=[] #Devide 300frame into 10 sectors and choose 1 frame from each sector
        
        for i in range(10):
            x.append(sample[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        
        X=torch.tensor(np.array(x)).unsqueeze(0).to(DEVICE)
        y=torch.tensor(test_y_label[idx]).to(DEVICE)
    
        logit = model(X)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy
    """"

In [ ]:
model = LSTM(1, 256, 75, 2, 0.2).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_loader)
    val_loss, val_accuracy = evaluate(model, test_loader)

    print("[이폭: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f" % (e, val_loss, val_accuracy))
    
    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/actionrecognition.pt')
        best_val_loss = val_loss

In [ ]:
#model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_loader)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

## Generator

1 * 30-> 1 * 30 * 75

In [12]:
class Discriminator(nn.Module):
    #def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
    def __init__(self, n_layers, hidden_dim, input_shape, n_classes, dropout_p=0.2):
        super(Discriminator, self).__init__()
        print("Building Basic Discriminator model...")
        self.n_layers = n_layers 
        #self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(input_shape, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, n_classes)
        self.out = nn.Softmax()
    def forward(self, x):
        #x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        c_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.lstm(x, (h_0, c_0))  # [i, b, h]
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.fc(h_t)  # [b, h] -> [b, o]
        logit = self.out(logit)
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [13]:
class Generator(nn.Module):
    #def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
    def __init__(self, n_layers, hidden_dim, input_shape, dropout_p=0.2):
        super(Generator, self).__init__()
        print("Building Basic generator model...")
        self.n_layers = n_layers 
        #self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(input_shape, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        #self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        #x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        c_0 = self._init_state(batch_size=x.size(0))
        x, (h, c) = self.lstm(x, (h_0, c_0))  # [i, b, h]
        
        return x
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [14]:
D = Discriminator(1, 256, 75, 2, 0.2).to(DEVICE)
G = Generator(1, 75, 1, 0.2).to(DEVICE)

d_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr)

Building Basic Discriminator model...
Building Basic generator model...


In [15]:
criterion=nn.BCELoss()

In [16]:
input_data=torch.randn(30,1)
input_data=torch.tensor(np.array(input_data)).unsqueeze(0).to(DEVICE)

In [17]:
data_loader=dl.GAN_dataloader("/home/dhk1349/Desktop/Capstone Design2/ntu/xsub/")
EPOCHS=100

In [19]:
total_step = len(data_loader)
for epoch in range(EPOCHS):
    for i, data in enumerate(data_loader):
        x = np.array(data[0][0])
        
        X=[]
        for i in range(10):
            X.append(x[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        
        X=torch.tensor(np.array(X)).unsqueeze(0).to(DEVICE)
        
        # '진짜'와 '가짜' 레이블 생성
        real_labels = torch.tensor([[0,1]]).to(torch.float32).to(DEVICE)
        fake_labels = torch.tensor([[1,0]]).to(torch.float32).to(DEVICE)
        
        # 판별자가 진짜 이미지를 진짜로 인식하는 오차를 예산
        outputs = D(X)
        
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        #print(real_score)
        # 무작위 텐서로 가짜 이미지 생성 
        z = torch.randn(30,1)
        z = torch.tensor(np.array(z)).unsqueeze(0).to(DEVICE)
        fake_images = G(z)
        
        # 판별자가 가짜 이미지를 가짜로 인식하는 오차를 계산
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # 진짜와 가짜 이미지를 갖고 낸 오차를 더해서 판별자의 오차 계산
        d_loss = d_loss_real + d_loss_fake

        # 역전파 알고리즘으로 판별자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # 생성자가 판별자를 속였는지에 대한 오차를 계산
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)
        
        # 역전파 알고리즘으로 생성자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    # 학습 진행 알아보기
    print('Epoch [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
          .format(epoch, EPOCHS, d_loss.item(), g_loss.item(), 
                  real_score.mean().item(), fake_score.mean().item()))
    if(epoch%3==0):
        torch.save(G.state_dict(), './snapshot/lstm_gan_'+str(epoch)+'.pt')

<ipython-input-12-82b9017cfe17>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logit = self.out(logit)


Epoch [0/100], d_loss: 0.0003, g_loss: 8.1092, D(x): 0.50, D(G(z)): 0.50
Epoch [1/100], d_loss: 0.0003, g_loss: 8.2174, D(x): 0.50, D(G(z)): 0.50
Epoch [2/100], d_loss: 0.0001, g_loss: 8.9858, D(x): 0.50, D(G(z)): 0.50
Epoch [3/100], d_loss: 0.0001, g_loss: 8.9690, D(x): 0.50, D(G(z)): 0.50
Epoch [4/100], d_loss: 0.0001, g_loss: 9.2898, D(x): 0.50, D(G(z)): 0.50
Epoch [5/100], d_loss: 0.0001, g_loss: 9.3278, D(x): 0.50, D(G(z)): 0.50
Epoch [6/100], d_loss: 0.0001, g_loss: 9.2489, D(x): 0.50, D(G(z)): 0.50
Epoch [7/100], d_loss: 0.0002, g_loss: 8.6667, D(x): 0.50, D(G(z)): 0.50
Epoch [8/100], d_loss: 0.0000, g_loss: 11.2961, D(x): 0.50, D(G(z)): 0.50
Epoch [9/100], d_loss: 0.0000, g_loss: 12.6177, D(x): 0.50, D(G(z)): 0.50
Epoch [10/100], d_loss: 0.0000, g_loss: 12.3034, D(x): 0.50, D(G(z)): 0.50
Epoch [11/100], d_loss: 0.0001, g_loss: 11.4435, D(x): 0.50, D(G(z)): 0.50
Epoch [12/100], d_loss: 0.0000, g_loss: 12.7569, D(x): 0.50, D(G(z)): 0.50
Epoch [13/100], d_loss: 0.0000, g_loss: 13.

In [25]:
model=Generator(1, 75, 1, 0.2)
model.load_state_dict(torch.load('./snapshot/lstm_gan_99.pt'))
for i in range(10):
    z = torch.randn(30,1)
    z = torch.tensor(np.array(z)).unsqueeze(0)
    print(z[0][0])
    output=model(z)
    #np.save("./snapshot/1105lstm_out_"+str(i)+".npy", output.detach().numpy().reshape(1,30,75))

Building Basic generator model...
tensor([1.1201])
tensor([-1.0182])
tensor([-0.1795])
tensor([0.8854])
tensor([-1.3263])
tensor([-1.7696])
tensor([-0.1415])
tensor([-0.0017])
tensor([1.3857])
tensor([-0.0093])


In [33]:
for j, data in enumerate(data_loader):
    
    if j==10:
        break
    x = np.array(data[0][0])
        
    X=[]
    for i in range(10):
        X.append(x[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        
    X=torch.tensor(np.array(X)).unsqueeze(0)
    print(X.shape)
    print(j)
    np.save("./realdata_numpy/r_"+str(j)+".npy", X.detach().numpy().reshape(1,10,75))

torch.Size([1, 10, 75])
0
torch.Size([1, 10, 75])
1
torch.Size([1, 10, 75])
2
torch.Size([1, 10, 75])
3
torch.Size([1, 10, 75])
4
torch.Size([1, 10, 75])
5
torch.Size([1, 10, 75])
6
torch.Size([1, 10, 75])
7
torch.Size([1, 10, 75])
8
torch.Size([1, 10, 75])
9
